# AMLD workshop by L2F – Learn to Forecast

Welcome to the "Machine Learning Competition: Tennis Analysis" workshop at AMLD 2019, organized by [L2F – Learn to Forecast](https://www.l2f.ch/)!

Starting from Jeff Sackman's crowdsourced [Match Charting Project](https://github.com/JeffSackmann/tennis_MatchChartingProject) dataset, we have put together for you a dataset containing detailed shot-by-shot descriptions of a large collection of points played between the Swiss tennis star Roger Federer and his historical rival Rafael Nadal (RN). The information recorded includes the type of shot, direction of shot, depth of returns, types of errors, and more. This is better explained below; we have also put together some preliminary data preparation to help you get started.

## Goal

The goal of this workshop is for you to come up with an optimized Phederos – the god of tennis – who will be able to destroy his ancient human rival. Do you think you can beat the beast?

From a more technical perspective, we ask you to provide a strategy function which, given an opponent's shot, will return what you believe to be an optimal response shot. Your answer must be in the form of a JSON dictionary as follows: each key is an opponent's shot, and the corresponding value is a list from which we will uniformly sample your response.

For more clarity, let us consider an example of such a strategy (the strings denote shot types which are better explained below):

*strat* = {'f3': ['b2', 'b2', 'b3', 'v1'], 'f1': ['f2'], ...}.

The keys in this dictionary are shots RN can produce. Suppose RN is performing a forehand on your backhand (the code for this shot is 'f3'); *strat* would reply to this shot in a *probabilistic* way as follows: 
- 'b2' with probability 1/2,
- 'b3' with probability 1/4,
- 'v1' with probability 1/4.

On the other hand, the same strategy replies *deterministically* to 'f1' (which is a forehand on our forehand), since we say 'f2' (a forehand straight to RN) with probability 1. In this way, your strategy can incorporate simple deterministic and more complex probabilistic decision-making in an arbitrary way.

## Testing environment

You have to beat a stochastic model which replies to your shots based on past statistics, but never replies with shots which have historically occurred too rarely. More precisely, this simulation environment will react based on the counting statistics of outcomes to (human) Federer's shots. By 'outcome' here we mean either:
- a simple reply shot or serve by historical RN (this is perhaps a bit confusing at first, but we consider a player's serve as a 'reply' to the opponent's 'waiting');
- a 'termination token', e.g. '*' for a winning shot, meaning that RN could not respond to your winning shot and the point terminates, or '@' indicating that with your shot you made an unforced error and lost the point – but note that such tokens can include more detailed information than this and be more than one character long;
- a shot by RN and a termination token, concatenated in a simple string, e.g., 'f1*' would indicate that our simulated RN replied with a winning forehand shot – ouch!

The Python code for this tester can be found in the last part of this notebook.

### Important remarks

- The simulation environment will prompt you with shots or termination/waiting tokens from a predetermined list, which can be accessed by loading the pickle file *possible_prompts.p*. Make sure you know how to react to each of the elements in this list!
- The pickle file *allowed_shots.p* contains a list which can be obtained from the list in *possible_prompts.p* by removing termination and waiting tokens. Replying with a string not in this list leads to losing of the point. 
- The "past statistics" in the stochastic model you will be fighting against come from a hold-out "test set" and **not** from the dataset you have access to. Make sure to build models which generalize well!
- We will impose a time constraint on the duration of the evalutation of your policy: we expect that your policy can be evaluated on an i7 quadcore 1000000 times in less than 3 minutes.

Will the true heroes of Machine Learning rise? :)

# Data from Jeff Sackmann's GitHub

As explained above, we provide you with a pandas dataframe (*dataframe.p*), taken from Jeff Sackmann's original dataset. 

Each row is one point:
- the variables 'Player1' and 'Player2' are the name of the players
- the 'match_id' variable identifies the tournament and date
- the varaible 'Shots' is a string containing all the shots, played alternatively from the players, during the point
- the variable 'Shots1' are the shots played only by RF
- the variable 'Shots0' contain the shots played only by RN
- The 'Outcome' variable describes the outcome of the point
- The 'Server' variable has boolean values: it is 1 in case RF is serving
- The 'Winner' variable has boolean values: it is 1 in case RF has won the point

Numbers are used to indicate direction and depth, while letters are used to specific shot types (e.g. 'f' stands for 'forehand') and error types ('n' stands for 'net'). The letter 'X' means that RN is waiting for RF to serve... don't let him wait too much! A few symbols are used for other purposes, such as types of errors (e.g. '@' means 'unforced error' and '+' indicates an approach shot).

To understand the codes for each shot, we prepared an explanation file named "Shot encodings" with more details. You can also check [this example video](https://drive.google.com/open?id=1Q4obFagfzxkFy_mq9C5gWITQiXxBQfCf) for a visual demonstration of this way of encoding.

In [1]:
# Importing libraries

import numpy as np
import pickle
import pandas as pd
from nltk import ngrams

In [14]:
# Historical data

df = pd.read_pickle('dataframe.p')
df.head(50)

,match_id,Player1,Player2,Shots,Shots1,Shots0,Outcome,Server,Winner
0,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 b;37 f1 f1,b;37 f1,4 f1,*,0,1
1,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,6 b3,b3,6,w#,0,0
2,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,6,,6,*,0,0
3,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,5 b18 b2 f1 b2 f1 s2 b+3,b18 f1 f1 b+3,5 b2 b2 s2,*,0,1
4,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 b27 f+3 b1,b27 b1,4 f+3,*,0,1
5,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 b2,4,b2,d#,1,1
6,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 f37 s1,4 s1,f37,n#,1,0
7,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 b28 b2 s3 f+1 b1,4 b2 f+1,b28 s3 b1,w#,1,1
8,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4,4,,*,1,1
9,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,5 f37 s1 b1,5 s1,f37 b1,w@,1,1


# Prompts and shots

We provide here the list of possible prompts and allowed shots – historically the most common shots – that you have to use. There are many more shots in the dataframe, but we only stick to the 100 shots provided and the 40 shots provided. It means that the historical RN/tester, will only shoot with one of the 100 prompts and you can reply using only one of the 40 shots.

In [10]:
# Loading the prompts and the shots

all_shots = pickle.load(open("allowed_shots.p", "rb"))
poss_prompts = pickle.load(open("possible_prompts.p" , "rb"))

print('Allowed shots: ', all_shots, '\n')
print('Possible prompts: ', poss_prompts)

len(poss_prompts)


Allowed shots:  ['4', '4+', '5', '6', '6+', 'X', 'b', 'b1', 'b17', 'b18', 'b19', 'b2', 'b27', 'b28', 'b29', 'b3', 'b37', 'b38', 'f', 'f+1', 'f+3', 'f1', 'f18', 'f2', 'f27', 'f28', 'f29', 'f3', 'f37', 'f38', 'r2', 's', 's1', 's2', 's28', 's3', 'v1', 'v3', 'z1', 'z3'] 

Possible prompts:  ['f3', 'f1', 'b2', 'b1', 'X', 'f2', 'b3', '4', '6', '5', '*', 'b28', 'f28', 'b27', 's2', 'f+3', 's3', 'n@', 'f+1', 'b18', 'd@', 'f38', 'b29', 's1', 'b38', 'f37', '#', 'w@', 'f27', 'f29', 'n#', 'd#', 'f', 'b', 's28', 'b37', 'b17', 'f1*', 'b19', 'b39', 'w#', 'r2', 'f18', 'f39', 'z3', 'v1', 'v3', 'f#', 's27', '4+', '6+', 'z1', 'b1*', 'f3*', 'm2', 'f17', 'r3', 's', 'r1', 's29', 's38', 'b+1', 'z2', 'o3', 'c6', 'v2', 'c4', 'b1w@', 'b#', 'b1n@', 'b+3', 'f19', 'b3*', 'f+2', 'f1w@', 'b3w@', 's37', 's39', 'f3n@', 'f3w@', 'm3', 'f1n@', 'b1d@', 'b2n#', 'f2d@', 'f1n#', 'b1w#', 's#', 'b1n#', 'x@', 'f1w#', 'y3', 'f2d#', 'b2d@', 'f2n#', 'f3d@', 'y1', 'b3w#', 'o1', 'f-3']


100

# Testing function for your model performances

See the file *aux_functions*.  The following function makes the historical RN become alive and play against you!

In [5]:
from aux_functions import *

In [6]:
Model, shots_opo_com = aux_fn(df, poss_prompts, all_shots)

In [7]:
# Building up the model function

#################### The threshold on shots rarity ###################
threshold = 1                                                        #
######################################################################

# Given current prompt p_t and shot s_t, return next prompt p_(t+1)
def model(prompt, shot):
    n_grams_ps = Model[poss_prompts.index(prompt), all_shots.index(shot)]
    if(len(n_grams_ps) > threshold):    # exclude rare reactions
        return np.random.choice(n_grams_ps)
    return 'n@'   # when (p, s) never happened, return unforced error in net

# Example: RN just served ('5') and we want to reply with a forehand ('f28'). This is what RN will reply (stochastic function)
model('5', 'f28')

'f3d@'

# Your turn to serve!

You can play against the historical RN: just type in the shots and see if you can beat him! You can try first with simple shots, like 'f3' or 'b2' or even with a volley 'v1'.

In [ ]:
current_prompt = 'X'
while True:       ###### CAREFUL WHEN INTERRUPTING
    shot = input()
    next_prompt = model(current_prompt, shot)
    print(next_prompt)
    current_prompt = next_prompt
    if is_win(next_prompt) == 1:
        print('Point won!')
        break
    if is_win(next_prompt) == 0:
        print('Point lost!')
        break

# Example and tester

We provide here an example of policy and a function with which you can test your model.

For testing your personal new policy, simply modify the first line of code and insert your policy dictionary. 

The final scoring will be done with this same function, with number_of_points_played >= 1000000 and different values of the threshold, to be sure that statistical fluctuations of the result are reduced.

In [8]:
# Test function that makes your policy (a.k.a. Phederos) play against historical RN !!

############################## Modify here! ##########################
example_policy = {'f3': ['b2', 'b2', 'b3', 'v1'], 'f1': ['f1']}
######################################################################

number_of_points_played = 150000


serves_opponent = [shot for shot in [sh[0] for sh in shots_opo_com] if shot[0] in '456']
points_won  = 0
points_lost = 0


for num in range(number_of_points_played):
    print(num, end = '\r')
    # Random choice on who is serving
    if np.random.randint(0,2) == 0 :
        current_prompt = 'X'
    else :
        current_prompt = np.random.choice(serves_opponent)
    while True:
        shot = np.random.choice(example_policy.get(current_prompt,['5']))
        next_prompt = model(current_prompt, shot)
        current_prompt = next_prompt
        if is_win(current_prompt) == 1:
            points_won  += 1
            break
        if is_win(current_prompt) == 0:
            points_lost += 1
            break
            
print('How well are we performing? '     , points_won/(points_lost + points_won))
print('Baseline historical performance: ', 1 - (df['Winner'].sum()/len(df['Winner'])))

How well are we performing?  0.04533333333333334
Baseline historical performance:  0.5030959752321982


# Your turn now!
Show us that you are up for the challenge and make your Phederos destroy the historical RN! Just be careful and do not get carried away... real challenges lie ahead of you all!

In [52]:
#df #['Shots']
#df['Server']
#for i in df['Shots', 'Server']
#    print(i)
#df['Shots', 'Server']
#df_i = df[['Shots','Server']]
all_shots = []
nadal = {}
for index,row in df.iterrows():
    server = row.Server
    outcome = row.Outcome
    winner = row.Winner
    shots = row.Shots.split(' ')
    if server == 1:  #Federer
        shots = ['x'] + shots
    
    
    for i in range(0,len(shots),2):
        try:
            all_shots.append([shots[i], shots[i+1], outcome, winner])
        except:
            all_shots.append([shots[i], 'Z', outcome, winner])

all_shots_df = pd.DataFrame(all_shots, columns=['Nadal','Federer', 'Outcome', 'Winner', 'Last'])
all_shots_df
print(shots)
    

['5', 's27', 'f+3', 'b2', 'v1', 'f-1']


In [53]:
#Under development

nadal_strat = {}
federer_strat = {}
for index, row in df.iterrows():
    server = row.Server
    shots = row.Shots.split(' ')
    print(shots)
    current = server
    for i, shot in enumerate(shots):
        if current == 0  #Nadal
            nadal_strat[]
        else:
            
    
    

['4', 'b;37', 'f1', 'f1']
['6', 'b3']
['6']
['5', 'b18', 'b2', 'f1', 'b2', 'f1', 's2', 'b+3']
['4', 'b27', 'f+3', 'b1']
['4', 'b2']
['4', 'f37', 's1']
['4', 'b28', 'b2', 's3', 'f+1', 'b1']
['4']
['5', 'f37', 's1', 'b1']
['6', 'b2']
['6', 'f']
['5', 'b28', 'f3', 'b2', 'b;2', 'f1', 'b1', 'f1']
['4', 's38', 'f1', 'b1', 'f1', 'b+3', 's3']
['6', 'b27', 'f+3', 'f3', 'z1', 'm3', 'o3']
['4', 'b29', 'f1', 's2', 'f3', 'f3', 'b2', 'f2', 'b2', 'b2', 'f1', 'b2', 'b2', 'b1', 'f1']
['4', 'f238', 'f2', 'f3', 'b3', 'f3']
['4', 'b28', 'f3']
['c6', 's']
['4', 'f17', 'b1', 'f3', 'f2', 'f1', 'b2']
['6']
['6', 'b28', 'f3', 'b2']
['4', 'b28', 'f1', 'f2', 'b3', 'b2']
['4', 'f18', 'b3']
['6']
['6', 'f']
['4']
['6']
['4', 'f1']
['5', 'f38', 'f3']
['6', 'b28', 'f3', 's2', 'f3', 'b2', 'b1', 'f3']
['4', 'b28', 'b1', 'f2', 'f3', 's2', 'f1']
['4', 'f18', 'b2', 'b2', 'b3', 'b3', 'f3', 'b2', 'b1', 'f1', 'b1', 'f3']
['4']
['6']
['4', 'f38', 'f3', 'f2', 'f1']
['4', 'b28', 'f3']
['4', 'f19']
['6', 'f1']
['4', 'f28', 'f1'

['6', 'b1']
['6', 'b19', 'b1', 'f1', 'b2', 'f2', 'b3', 'y+1']
['6']
['6', 'f28', 'b2', 'f3', 'f3', 'y+2', 's1', 'r3']
['4', 'b28', 'b1', 'f3', 'f3']
['4', 'b18', 'b3', 'b3', 'f3']
['5', 'b27', 'f3', 'b2', 'f3', 'b2', 'f3', 'b3', 'f3', 's2', 'f1', 'f1', 'b3', 'b2', 'f1']
['6', 'f']
['4', 'b28', 'f2']
['4', 'b38', 'b2', 'f3']
['6+', 'm17', 'o3']
['5', 'b3']
['6', 'b18', 'b3']
['6', 'b2']
['5', 'b28', 'f3', 'b1']
['4', 'b37', 'f1', 'f3']
['5', 'f28', 'f1', 'b2', 'f1', 'b2', 'f3', 'f3', 'y1']
['4', 'f38', 'f1']
['6', 'f18', 'f1', 'b3', 'f2', 'f3', 'b2', 'f2', 'f1', 'b3', 'b1', 'b3', 'b3', 'f2', 'f2', 'b3', 'b2', 'b3', 'b;2', 'b;3', 'f2', 'f1', 'f1', 'b3']
['6']
['6+', 'b27', 'z1']
['6']
['6', 'f28', 'f1', 'b3', 'b1', 's2']
['6', 's29', 'f1', 'b2', 'f3', 'f3', 'b1', 'b1', 'f2', 'f3', 'b2']
['4']
['6+', 's27', 'i1', 'b2', 'b2', 'b3', 'b3']
['6', 'f;38', 'f1']
['4', 's37', 'f+3', 'b1', 'z1']
['6', 'b18', 's2', 'b3', 'f3', 'b1', 's2', 'f3', 'f3', 'b3', 'f1', 'f1', 'b3', 'f2', 'f2', 'b3', 'f1',

['4+', 's3', 'v1', 'b']
['6', 'b37', 'b1']
['6', 'b18', 's1', 'f1', 'b1', 'f3', 'f3', 's']
['4', 'f17', 's1', 'f3', 'f3', 'b2', 'b3', 's1', 'b1', 'f3', 'f3']
['6', 'f19', 'b3', 'f1', 'b1']
['5', 's29', 'f3', 'b3', 'f3', 'b3']
['6', 'f27', 'f+3', 'b1', 'v1', 'b1', 'v3']
['6']
['4', 'b37', 'b3', 'f']
['4', 'f']
['6', 's29', 'f3', 'b3']
['4', 'b28', 'b1', 'f3']
['4', 'b27', 'f1', 'f1', 'b3']
['4', 's']
['c4']
['4']
['5', 'b39', 'b3', 'f3', 'b+1']
['6', 'b;39', 'b3']
['4', 'b3', 'z3', 'm3', 'p3']
['6', 'b28', 'f+1', 'm', 'o1', 'b3', 'b3', 'f1', 'v1']
['6', 's']
['6', 'b27', 'f+3', 'm2', 'o3']
['6', 'f27', 'f3', 'b1', 'f1']
['6', 'f29', 'f1', 'b2', 'f1', 'b1']
['4', 'f39', 'b1', 'b3', 'b3', 'f3', 'b;1', 's1', 'v1', 'b3']
['4', 'f39', 'b2', 'f3']
['6']
['6']
['6', 's28', 'f3']
['5']
['4', 'b19', 'b1', 'f3', 'f3', 'b3', 'f1', 'f1', 'b3']
['6', 'b37', 'f2', 'f3', 'f3', 'f3', 'b2', 'b3', 'b3', 'f3']
['4']
['6', 'b']
['6+', 'f', 'v1']
['6', 'b28', 'f+1', 'b1']
['6', 'b19', 'b1', 'f3']
['4', 'b29

['5', 'f1']
['5', 'b38', 'f3', 's2', 's+2', 'm2', 'o3']
['5', 'f27', 'f+1', 'b3']
['4', 'f38', 'f1', 's1']
['5', 'f28', 'f2', 'f1', 'b2', 'f3']
['4', 'b29', 'f3', 'b1', 'b3', 'b3', 'f3', 'b1', 'b1', 'f2', 'f3', 's2', 'f3', 'b2', 'f3', 'b3', 'u1']
['5', 'f18', 'b1']
['4']
['c5', 'b28', 'f2', 'f3']
['4', 'b3']
['4', 'b27', 'f3', 'b2', 'f2']
['5']
['4', 'b28', 'f3', 'b1']
['4', 'b28', 'f2', 'f2', 'f2', 'b3', 's3', 'f3', 's1', 'b1', 'f1', 'y3', 's2']
['6', 'b38', 'f3', 'f3']
['6', 'f3']
['4', 'f38', 'f1', 'b1', 'f3', 'f']
['5', 'f28', 'f1', 'b1', 'f1', 'b2', 'f2', 'f3', 'y3']
['5', 'b28', 'f+1', 'r2', 'i3', 's3', 'f1', 'r3']
['5', 'f18', 'b3']
['5', 'f27', 'f+1', 'b2', 'i2', 'b1', 'v1']
['4', 'f28', 'b1', 'f1', 'b2', 'f2', 'f3', 's3', 'u1', 'f1', 'z2', 's3']
['4', 'f38', 'f2', 'u2', 'f3', 'f3']
['4', 'b28', 'f3', 'f2', 'v2', 'b2']
['4', 'b28', 'b3', 'b3', 'f3', 'b2', 'f3', 's2', 'f+3', 'b2', 'v1']
['5', 'b28', 'f2', 'f1', 'b2', 'f+3', 'r2', 'z1']
['4', 'f']
['4']
['4', 'b17', 'b1']
['6', '